In [14]:
# Import packages
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score


In [2]:
df = pd.read_csv("football_players.csv",  encoding='latin1')

In [3]:
df.head()

Name  Age Nationality  Overall  Acceleration  Aggression  \
0  Cristiano Ronaldo   32    Portugal     94.0          89.0        63.0   
1           L. Messi   30   Argentina     93.0          92.0        48.0   
2             Neymar   25      Brazil     92.0          94.0        56.0   
3          L. Suárez   30     Uruguay     92.0          88.0        78.0   
4           M. Neuer   31     Germany     92.0          58.0        29.0   

   Agility  Balance  Ball control  Composure  ...  Shot power  Sliding tackle  \
0     89.0     63.0          93.0       95.0  ...        94.0            23.0   
1     90.0     95.0          95.0       96.0  ...        85.0            26.0   
2     96.0     82.0          95.0       92.0  ...        80.0            33.0   
3     86.0     60.0          91.0       83.0  ...        87.0            38.0   
4     52.0     35.0          48.0       70.0  ...        25.0            11.0   

   Sprint speed  Stamina  Standing tackle  Strength  Vision  Volleys  \
0          91.0     92.0             31.0      80.0    85.0     88.0   
1          87.0     73.0             28.0      59.0    90.0     85.0   
2          90.0     78.0             24.0      53.0    80.0     83.0   
3          77.0     89.0             45.0      80.0    84.0     88.0   
4          61.0     44.0             10.0      83.0    70.0     11.0   

   Preferred Positions  Preferred Positions Type  
0               ST LW                    Forward  
1                  RW                    Forward  
2                  LW                    Forward  
3                  ST                    Forward  
4                  GK                 GoalKeeper  

[5 rows x 40 columns]

In [4]:
# Finding the Most Common Overall Score

most_common_score = df['Overall'].mode()[0]
print(f"The most common Overall score is: {most_common_score}")

The most common Overall score is: 66.0


In [5]:
# Subsetting Players Who Can Play in Central Defence

central_defenders = df[df['Preferred Positions'].str.contains('CB')].copy()

In [6]:
# Splitting the Group into Categories

# Step 4: Splitting the Group into Categories
def classify_player(overall):
    if overall >= 80:
        return 'World Class'
    elif 70 <= overall < 80:
        return 'Good'
    else:
        return 'Mediocre'

# Apply the classification function using .loc
central_defenders.loc[:, 'Class'] = central_defenders['Overall'].apply(classify_player)

# Display the first few rows of the updated DataFrame
print(central_defenders.head())

            Name  Age Nationality  Overall  Acceleration  Aggression  Agility  \
10  Sergio Ramos   31       Spain     90.0          75.0        84.0     79.0   
17  G. Chiellini   32       Italy     89.0          68.0        92.0     59.0   
24    L. Bonucci   30       Italy     88.0          62.0        82.0     60.0   
25    J. Boateng   28     Germany     88.0          72.0        82.0     58.0   
26      D. Godín   31     Uruguay     88.0          62.0        86.0     63.0   

    Balance  Ball control  Composure  ...  Sliding tackle  Sprint speed  \
10     60.0          84.0       80.0  ...            91.0          77.0   
17     64.0          57.0       82.0  ...            90.0          78.0   
24     52.0          75.0       84.0  ...            88.0          72.0   
25     53.0          71.0       86.0  ...            90.0          78.0   
26     58.0          76.0       82.0  ...            89.0          67.0   

    Stamina  Standing tackle  Strength  Vision  Volleys  Prefe

In [7]:
# Building a Random Forest Classifier

# Prepare the data

# Select only numeric columns
numeric_columns = central_defenders.select_dtypes(include=['number']).columns

X = central_defenders[numeric_columns]
y = central_defenders['Class']

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3630 entries, 10 to 17980
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 3630 non-null   int64  
 1   Overall             3630 non-null   float64
 2   Acceleration        3630 non-null   float64
 3   Aggression          3630 non-null   float64
 4   Agility             3630 non-null   float64
 5   Balance             3630 non-null   float64
 6   Ball control        3630 non-null   float64
 7   Composure           3630 non-null   float64
 8   Crossing            3630 non-null   float64
 9   Curve               3630 non-null   float64
 10  Dribbling           3630 non-null   float64
 11  Finishing           3630 non-null   float64
 12  Free kick accuracy  3630 non-null   float64
 13  GK diving           3630 non-null   float64
 14  GK handling         3630 non-null   float64
 15  GK kicking          3630 non-null   float64
 16  GK positi

In [9]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=911)

In [10]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
# Build the model
model = RandomForestClassifier(n_estimators=500, random_state=1971)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=500, random_state=1971)

In [13]:
# Get feature importances
importances = model.feature_importances_
feature_names = X.columns
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

print("Top 10 features:")
print(feature_importances.head(10))

Top 10 features:
             Feature  Importance
1            Overall    0.375805
32   Standing tackle    0.122604
23           Marking    0.100337
19     Interceptions    0.094376
29    Sliding tackle    0.066217
26         Reactions    0.055453
18  Heading accuracy    0.037714
7          Composure    0.033208
3         Aggression    0.019799
21      Long passing    0.013203


In [16]:
# Initialize a dictionary to store F1 scores
f1_scores = {}

# Train and evaluate KNN models with k varying from 1 to 5
for k in range(1, 6):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    
    # Calculate F1 scores for each class
    f1_world_class = f1_score(y_test, y_pred, labels=['World Class'], average='weighted')
    f1_good = f1_score(y_test, y_pred, labels=['Good'], average='weighted')
    
    # Store the F1 scores
    f1_scores[k] = {'World Class': f1_world_class, 'Good': f1_good}

# Determine the best k for each group
best_k_world_class = max(f1_scores, key=lambda k: f1_scores[k]['World Class'])
best_k_good = max(f1_scores, key=lambda k: f1_scores[k]['Good'])

print(f"Best k for World Class: {best_k_world_class}")
print(f"Best k for Good: {best_k_good}")

# Print the F1 scores for each k
for k, scores in f1_scores.items():
    print(f"k={k}: World Class F1 Score={scores['World Class']}, Good F1 Score={scores['Good']}")

Best k for World Class: 1
Best k for Good: 5
k=1: World Class F1 Score=0.6666666666666666, Good F1 Score=0.7840909090909091
k=2: World Class F1 Score=0.4680851063829788, Good F1 Score=0.817283950617284
k=3: World Class F1 Score=0.5925925925925926, Good F1 Score=0.8326300984528833
k=4: World Class F1 Score=0.4680851063829788, Good F1 Score=0.8496732026143791
k=5: World Class F1 Score=0.5925925925925926, Good F1 Score=0.8511235955056181
